In [ ]:
!pip install simpy

In [ ]:
import simpy
import random
import statistics

wait_times = []

class network(object):
    def __init__(self, env, num_servers_1, num_servers_2, service_rate_1, service_rate_2):
        self.env = env
        self.server_1 = simpy.Resource(env, num_servers_1)
        self.server_2 = simpy.Resource(env, num_servers_2)
        self.service_rate_1 = service_rate_1
        self.service_rate_2 = service_rate_2

    def Queue_1(self, customer):
        yield self.env.timeout(random.expovariate(self.service_rate_1))

    def Queue_2(self, customer):
        yield self.env.timeout(random.expovariate(self.service_rate_1))


def go_to_queues(env, customer, network):
    arrival_time = env.now

    with network.server_1.request() as request:
        yield request
        yield env.process(network.Queue_1(customer))


    with network.server_2.request() as request:
        yield request
        yield env.process(network.Queue_2(customer))

    wait_times.append(env.now - arrival_time)


def run_queues(env, arrival_rate, num_servers_1=1, num_servers_2=1, service_rate_1=1, service_rate_2=1):
    net = network(env, num_servers_1, num_servers_2, service_rate_1, service_rate_2)

    while True:
        yield env.timeout(random.expovariate(arrival_rate))

        customer =+ 1
        env.process(go_to_queues(env, customer, net))


def get_average_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)
    return average_wait


def main(arrival_rate=1, service_rate_1=2, service_rate_2=2, num_servers_1=1, num_servers_2=1):
    if arrival_rate >= service_rate_1*num_servers_1 or arrival_rate >= service_rate_2*num_servers_2:
        print("for these rates the system isn't stable in steady state.")

    # Run the simulation
    env = simpy.Environment()
    env.process(run_queues(env, arrival_rate, num_servers_1, num_servers_2, service_rate_1, service_rate_2))
    env.run(until=10**5)

    # View the results
    ave = get_average_wait_time(wait_times)
    print(
        "Running simulation...",
        f"\nThe average wait time is {ave}.",
    )

In [ ]:
main()
